In [ ]:
from semconstmining.declare.enums import Template
import logging
import pandas as pd
from semconstmining.mining.extraction.extractionhandler import DeclareExtractor
from semconstmining.parsing.label_parser.nlp_helper import NlpHelper
from pathlib import Path
import os

from semconstmining.config import Config
from semconstmining.main import get_resource_handler

logging.basicConfig(format='[%(asctime)s] p%(process)s {%(filename)s:%(lineno)d} %(levelname)s - %(message)s',
                    level=logging.WARNING)

MODEL_COLLECTION = "#####"

conf = Config(Path(os.getcwd()).parents[1].resolve(), MODEL_COLLECTION)
conf.CONSTRAINT_TYPES_TO_IGNORE = [ct for ct in conf.CONSTRAINT_TYPES_TO_IGNORE if ct not in [Template.INIT.templ_str, Template.END.templ_str, Template.CHOICE.templ_str]]
print(conf.CONSTRAINT_TYPES_TO_IGNORE)

nlp_helper = NlpHelper(conf)
resource_handler = get_resource_handler(conf, nlp_helper)
extractor = DeclareExtractor(config=conf, resource_handler=resource_handler)
dfs = [extractor.discover_plain_control_flow_constraints(t, no_events=True) for t in resource_handler.bpmn_logs.reset_index().itertuples()]
all_constraints = pd.concat(dfs).astype({conf.LEVEL: "category"})

In [ ]:
resource_handler.bpmn_model_elements

In [ ]:
from tqdm import tqdm
from semconstmining.mining.aggregation.subsumptionanalyzer import SubsumptionAnalyzer
all_constraints[conf.SUPPORT] = 1
all_constraints[conf.OBJECT] = ""
activity_constraints = all_constraints
#activity_constraints[conf.CONSTRAINT_STR] = activity_constraints[conf.CONSTRAINT_STR].str.replace("|", "")
#activity_constraints[conf.CONSTRAINT_STR] = activity_constraints[conf.CONSTRAINT_STR].str.strip()
#activity_constraints = activity_constraints[[conf.MODEL_ID,conf.CONSTRAINT_STR]]
# Group by 'conf.MODEL_ID' and generate a list of 'conf.CONSTRAINT_STR' elements for each group
grouped = activity_constraints.groupby(conf.MODEL_ID)
dfs = []
for group_id, group in tqdm(grouped):
    if group_id == "1bd1ef6b99d6492c9d533f9120462f18":
        print(group[conf.CONSTRAINT_STR])
    # event_elms = resource_handler.bpmn_model_elements[(resource_handler.bpmn_model_elements[conf.MODEL_ID] == group_id) & (resource_handler.bpmn_model_elements[conf.ELEMENT_CATEGORY].str.contains("Event"))][conf.CLEANED_LABEL].values
    # event_elms = [e for e in event_elms if e != ""]
    # # print(event_elms)
    # consts = group[(~group[conf.LEFT_OPERAND].isin(event_elms)) & (~group[conf.RIGHT_OPERAND].isin(event_elms))]
    # if len(consts) < len(group):
    #     print(group_id + ": Skipping " + str(len(group) - len(consts)) + "constraints due to event labels")
    subsumption_analyzer = SubsumptionAnalyzer(conf, group)
    subsumption_analyzer.check_refinement()
    subsumption_analyzer.check_subsumption()
    subsumption_analyzer.check_equal()
    dfs.append(subsumption_analyzer.constraints[~subsumption_analyzer.constraints[conf.REDUNDANT]])
refined = pd.concat(dfs)
refined

In [ ]:
grouped = refined.groupby(conf.MODEL_ID).agg({conf.CONSTRAINT_STR: list})

# Rename the 'conf.CONSTRAINT_STR' column to 'constraints'
grouped = grouped.rename(columns={conf.CONSTRAINT_STR: 'constraints'})

# Reset the index to turn the 'conf.MODEL_ID' column into a regular column
grouped = grouped.reset_index()

In [ ]:
grouped["constraints"].values[2]

In [ ]:
grouped.to_pickle(MODEL_COLLECTION + "_model_id_to_constraints.pkl")